In [3]:
!pip install textblob
!pip install nltk

In [7]:
!python -m textblob.download_corpora
!nltk.download('stopwords')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.
/bin/bash: -c: line 1: syntax error near unexpected token `'stopwords''
/bin/bash: -c: line 1: `nltk.download('stopwords')'


In [11]:
import requests
import base64
from textblob import TextBlob
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter

def extract_music_keywords(text):
    # Tokenize the text
    blob = TextBlob(text)
    words = [word.lower() for word in blob.words if word.isalpha()]

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word not in stop_words]

    # Extract keywords
    keywords = Counter(filtered_words)

    # Define music-related keywords based on scene context
    music_keywords = {
        'upbeat': ['cheerful', 'smile', 'joyful', 'energetic'],
        'relaxing': ['calm', 'comfort', 'peaceful', 'serene'],
        'melancholic': ['sad', 'melancholy', 'wistful', 'nostalgic'],
        'intense': ['tense', 'dramatic', 'powerful', 'emotional']
    }

    # Find the most frequent keywords that match the music-related context
    matched_keywords = Counter()
    for mood, mood_keywords in music_keywords.items():
        matched_keywords[mood] += sum(keywords[key] for key in keywords if key in mood_keywords)

    # Return the most frequent mood keywords
    top_moods = matched_keywords.most_common(3)

    return [mood for mood, _ in top_moods]

# Example text input
text = """
The story of a father who neglects his responsibilities as the head of the family, a mother who thinks finding the bridegrooms for their daughters is the only goal in her life. Pride and Prejudice written by Jane Austen is narrated in the point of view of the key character of Elizabeth Bennet, the second of the five daughters. Elizabeth’s courtship with Fitzwilliam Darcy, a wealthy and eligible bachelor who owns a family estate is narrated by the author in a sequence of incidents which deal with 19th century England social issues of manners, morality and education."""



# Get music-related keywords
music_keywords = extract_music_keywords(text)
print("Music-related keywords:", music_keywords)

length = 8  # Length of music clip in seconds
keyword = music_keywords

resp = requests.post(
    "https://model-232pg60q.api.baseten.co/production/predict",
    headers={"Authorization": "Api-Key cdwwufG4.wYPONrLlil3QcVDmHbdz2JSRJcJ2YLnU"},
    json={'prompts': keyword, 'duration': length},
)

print(resp.json())

# Convert the base64 output to an audio file
res = resp.json()
output = res.get("data")
for idx, clip in enumerate(output):
    with open(f"musicgen_output_{idx}.wav", "wb") as f:
        f.write(base64.b64decode(clip))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Music-related keywords: ['upbeat', 'relaxing', 'melancholic']
{'data': ['...']}


Combine the music with the video

In [ ]:
# put the music into the video
from moviepy.editor import VideoFileClip, AudioFileClip

def add_audio_to_video(video_path, audio_path, output_path):
    try:
        # Load video and audio clips
        video_clip = VideoFileClip(video_path)
        audio_clip = AudioFileClip(audio_path)

        # Set audio clip duration to match video duration
        audio_clip = audio_clip.set_duration(video_clip.duration)

        # Combine video clip with audio clip
        video_with_audio = video_clip.set_audio(audio_clip)

        # Write the new video with audio to the output path
        video_with_audio.write_videofile(output_path, codec="libx264", audio_codec="aac")

        print("Video with audio saved successfully at:", output_path)

    except Exception as e:
        print("An error occurred:", e)

# Example usage
video_path = "path_to_video.mp4"  # Replace with the path to your video file
audio_path = "path_to_audio.wav"  # Replace with the path to your audio file
output_path = r'C:\Users\weiduow-adm\output_video_with_audio.mp4'  # Output path for the new video with audio

add_audio_to_video(video_path, audio_path, output_path)